In [1]:
from langchain_core.tools import tool

In [ ]:
# Step 1 : Define The function

def multiply(a, b):
    """
    Multiplies two numbers.
    """
    return a * b

In [3]:
# Step 2 : add type hints

def multiply(a:int, b:int) -> int:
    """
    Multiplies two numbers.
    """
    return a * b

In [4]:
# Step 3 : add tool decorator

@tool
def multiply(a:int, b:int) -> int:
    """
    Multiplies two numbers.
    """
    return a * b

In [5]:
result = multiply.invoke({"a" : 3, "b":5})
print(result)

15


In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiplies two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Whenever we use the tool, we don't send the tool(or function) to LLM, rather we send this schema.

In [7]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiplies two numbers.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# Method Two : Using Structured Tool

A **Structured tool** in Lagcahin is a special type of tool where the input to the tool follows a structured schema, typically defined using a Pydantic model.

In [8]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [9]:
class MultiplyInput(BaseModel):
    a : int = Field(required=True, description="The first number to add")
    b : int = Field(required=True, description="The Second number to add") 

In [11]:
def multiply_func(a:int, b:int)->int:
    return a*b

In [12]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",
    description= "Multiply Two numbers",
    args_schema=MultiplyInput
)

In [13]:
result = multiply_tool.invoke({"a":3, "b":4})

print(result)

12


In [15]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply Two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The Second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# Method Three : Using BaseTool

**BaseTool** is the abstract base class for all tools in LangChain. It defines the core structure and interface that ny tool must follow, whether it's a simple one-liner or a fully customized function.

 - All other tool types like ``@tool``,``StructuredTool`` are built on top of BaseTool

In [16]:
from langchain.tools import BaseTool
from typing import Type 

In [17]:
# args schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The Second number to add")

In [18]:
class MultiplyTool(BaseTool):
    name:str = "multiply"
    description:str = "Multiply Two numbers"
    args_schema : Type[BaseModel] = MultiplyInput
    
    def _run(self, a:int, b:int) -> int:
        return a*b

In [19]:
multiply_tool = MultiplyTool()

In [20]:
result = multiply_tool.invoke({'a':3, 'b':3})
print(result)

9


# Toolkit

A toolkit is just a collection (bundle) of related tools that serve a common purpose - packaged together for convenience and **reusability**.

In LangChin a toolkit might be ``GoogleDriveToolKit``,and it might contain following tools:
 - ``GoogleDriveCreateFileTool`` : Upload a file

 - ``GoogleDriveSearchTool`` : Search for a file by name/content
 
 - ``GoogleDriveReadFileTool`` : Read content of a file

Here we'll build a simple toolkit of math related things  

In [22]:
from langchain.tools import tool

# Custom tools
@tool
def add(a:int, b:int) -> int:
    """add two numbers"""
    return a+b
    
    
@tool
def multiply(a:int, b:int) -> int:
    """multiply two numbers"""
    return a*b

In [23]:
class MathToolKit:
    def get_tools(self):
        return [add, multiply]

In [24]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => add two numbers
multiply => multiply two numbers
